# Evaluation of classification methods for automatic phytolith identification

This notebook contains functions and examples to train and evaluate several classifiers using cross validation.

The execution of this notebook with a great number of images can take several hours, depending on the machine where it is executed.

## Authors
- José-Francisco Díez-Pastor
- Pedro Latorre-Carmona
- Álvar Arnaiz-González
- Javier Ruiz-Pérez
- Débora Zurro

# Perform experiments and examine results


In [ ]:
'''
Import libraries
'''
import pandas as pd
import numpy as np
import pickle
import os

from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
'''
Configuration data
'''

path = "./phytoliths"

features_file = path + os.sep + "features_2020.csv"

In [ ]:
# Load the features dataset
df = pd.read_csv(features_file,index_col=0)

y = df.Class.values

# Choose only 'Elliptic Fourier Descriptor' feature names
efd_cols = df.columns.str.startswith("edf")

# EFD Dataset
X_efd = df[df.columns[efd_cols]].values

# Choose only basic geometric feature names
morpho_cols = ["Perimeter","PerimeterHull","Area","Convex Area","Major axis length", 
               "Minor axis length","Equivalent diameter","Form factor","Length",
               "Width","Convexity","Solidity", "AspectRatio","Roundness","Compactness"]

# Morphological and geometic features Dataset
X_morpho = df[morpho_cols].values

# All features dataset
X_all = np.concatenate((X_morpho, X_efd), axis=1)

In [ ]:
# Show the shape of Elliptic Fourier Descriptor dataset
X_efd.shape

In [ ]:
'''
List of datasets and their names included in the experimental study
'''

datasets = [(X_morpho,y), (X_efd,y), (X_all,y)]
dataset_names = ["Data Morpho", "Data EFD", "Data All"]

In [ ]:
'''
Definition of the SVM parameter search
'''
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid_svm = dict(gamma=gamma_range, C=C_range)
nested_cv = 5

grid_svm = GridSearchCV(SVC(), param_grid=param_grid_svm, cv=nested_cv)

In [ ]:
# Show the range of values to be explored
C_range,gamma_range

In [ ]:
'''
Definition of the MLP parameter search
'''
alpha_range = np.logspace(-5, -1, 5)
hidden_layer_sizes_range=[(50,),(100,),(200,),(500,),(1000,)]

param_grid_mlp = dict(alpha=alpha_range, hidden_layer_sizes=hidden_layer_sizes_range)


grid_mlp = GridSearchCV(MLPClassifier(max_iter=1000,
                                      early_stopping=True), param_grid=param_grid_mlp, cv=nested_cv)

In [ ]:
'''
List of classifiers and their names included in the experimental study
'''

cls_names = ["Nearest Neighbors", "SVM", "Decision Tree", "Random Forest","Gradient Boosting Trees","MLP"]

classifiers = [
    make_pipeline(StandardScaler(), KNeighborsClassifier(3)),
    make_pipeline(StandardScaler(), grid_svm),
    DecisionTreeClassifier(random_state=0),
    RandomForestClassifier(random_state=0, n_estimators=100),
    GradientBoostingClassifier(random_state=0, n_estimators=100),
    make_pipeline(StandardScaler(), grid_mlp)]

In [ ]:
def cross_validate_preds_model(X, y, model, num_folds):
        '''
        Perform cross validation with a model and a dataset (X and y),
        and returns the predictions to later obtain the measurements 
        you want
        
        Parameters
        ----------
        X: numpy.array
            Dataset (features)
        Y: numpy.array
            Dataset (Target)
        model: scikit_model
            model to be trained
        num_folds: int
            number of folds in the cross validation
        
        Return
        -------
        array 
            array of prediccions obtained using cross_validation
        '''
        print('\t'+str(model)[:20], end=' - ')
        preds = cross_val_predict(model,X,y,cv=num_folds)
        print('OK')
        
        return preds

In [ ]:
def run_all_save(num_folds,filename):
    '''
    Perform cross validation with all models and datasets.
        
        
    Parameters
    ----------
    num_folds: int
        number of folds in the cross validation
    filename: string
        name of the file that stores the predictions obtained using crossvalidation
        
    Return
    -------
    
    ''' 
    
    all_preds = {}

    for dataset,dataset_name in zip(datasets, dataset_names):
        print(dataset_name)
        X,y = dataset
        for model,cls_name in zip(classifiers,cls_names):
            print(cls_name)
            preds = cross_validate_preds_model(X, y, model, num_folds)
            all_preds[(dataset_name,cls_name)]=(y,preds)

    all_preds["cls_names"]=cls_names
    all_preds["dataset_names"]=dataset_names

    with open(filename, 'wb') as fp:
         pickle.dump(all_preds, fp)   

In [ ]:
'''
All the predictions are going to be saved in a Python dictionary for 
further analysis.
'''

filename3 = 'predicciones3_20.obj'
filename5 = 'predicciones5_20.obj'
filename10 = 'predicciones10_20.obj'

In [ ]:
# Run the experiments with cv=3 and save
run_all_save(3,filename3)

In [ ]:
run_all_save(5,filename5)

In [ ]:
run_all_save(10,filename10)

# Exploring the results

If the experiments have been done previously, you only need to execute from this part.

The results are going to be loaded from the hard disk

In [ ]:
import pickle
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

In [ ]:
def conf_mat_df(cm,labels):
    '''
    Create a confusion matrix in a DataFrame
        
        
    Parameters
    ----------
    cm: ndarray 2D
        confusion matrix
    labels: list
        List of class names
        
    Return DataFrame
    -------
    
    ''' 

    return (pd.DataFrame(cm,index=labels, columns=labels)
          .rename_axis("actual")
          .rename_axis("predicted", axis=1))

In [ ]:
%matplotlib inline
#https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/image_annotated_heatmap.html
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
filename3 = 'predicciones3_20.obj'
filename5 = 'predicciones5_20.obj'
filename10 = 'predicciones10_20.obj'

def get_results(filename):
    '''
    Load the file with the predictions.
    Compute accuracy, confusion matrix and other measures.
        
        
    Parameters
    ----------
    filename: string
        name of the file that stores the predictions obtained using crossvalidation
        
    Return
    dictionary
        A dictionary of key:values that asociates the name
        of a measure or chart with the value
    -------
    
    ''' 

    with open(filename, 'rb') as fp:
        all_preds = pickle.load(fp)

    cls_names = all_preds.pop("cls_names")
    dataset_names = all_preds.pop("dataset_names")

    data_cls_pairs = list(all_preds.keys())
    data_cls_pairs.sort()

    results = {}


    acc_df = pd.DataFrame(index=dataset_names, columns=cls_names)

    ## A DataFrame is created to store the accuracy in each clase
    for dataset in dataset_names:
        results[(dataset,"acc")] = pd.DataFrame(columns=cls_names)


    for dataset_name,cls_name in data_cls_pairs:

        #print(dataset_name,cls_name)
        y_true, y_pred = all_preds[(dataset_name,cls_name)]
        labels = list(np.unique(y_true))

        acc = accuracy_score(y_true, y_pred)
        # Fill accuracy dataframe
        acc_df.at[dataset_name,cls_name]=acc

        # Get conf_mat
        cm = confusion_matrix(y_true, y_pred)
        cm_df = conf_mat_df(cm,labels)
        results[(dataset_name,cls_name,"cm")] = cm_df

        # Get classification report
        report = classification_report(y_true, y_pred, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        results[(dataset_name,cls_name,"report")] = report_df

        # Acc per class
        cm_dig = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_dig = cm_dig.diagonal()

        dfi = results[(dataset_name,"acc")]
        dfi[cls_name]=pd.Series(cm_dig,labels)    
        results[(dataset_name,"acc")]=dfi.copy()


    results["Acc"] = acc_df
    return results
        
        
results5 = get_results(filename5)
results3 = get_results(filename3)
results10 = get_results(filename10)

result_dict = {3:results3,5:results5,10:results10}

### Visualize Results

In [ ]:
import copy
import ipywidgets as widgets
from IPython.display import clear_output

results = copy.deepcopy(result_dict[5])

dropdown_tables = widgets.Dropdown(
    options=list(results.keys())+["Select one table"],
    value="Select one table",
    description='Table:',
)

dropdown_folds = widgets.Dropdown(
    options=[3,5,10],
    value=5,
    description='Folds:',
)

lbl = widgets.Label(value="")


visor = widgets.Output(layout={'border': '1px solid black'})

def on_change_table(change):
    if change['type'] == 'change' and change['name'] == 'value':
        
        visor.clear_output()
        lbl.value = str(dropdown_folds.value)+" "+str(dropdown_tables.value)
        with visor:
            display(results.get(change['new']))
            
def on_change_folds(change):
    if change['type'] == 'change' and change['name'] == 'value':
        
        visor.clear_output()
        lbl.value = str(dropdown_folds.value)+" "+str(dropdown_tables.value)
        results = copy.deepcopy(result_dict[change['new']])
        with visor:
            display(results.get(dropdown_tables.value))
        
dropdown_tables.observe(on_change_table)
dropdown_folds.observe(on_change_folds)

box = widgets.VBox([lbl,dropdown_folds, dropdown_tables, visor])

display(box)

In [ ]:
df_total = results10["Acc"].astype(float)


df_conf = results10[("Data Morpho","SVM","cm")].astype(float)
df_report = results10[("Data Morpho","SVM","report")].astype(float)

In [ ]:
# df_total.round(4).to_latex()
df_total

In [ ]:
df_conf

In [ ]:
df_report.round(4)[["precision","recall","f1-score"]]

In [ ]:
'''
Heatmap accuracy
'''
rows = df_total.index
cols =  df_total.columns

metric = "Accuracy"

format_str = "{x:.4f}"

values = df_total.values



fig, ax = plt.subplots()
ax.grid(False)

im, cbar = heatmap(values, rows, cols, ax=ax,
                   cmap="Blues", cbarlabel=metric)
texts = annotate_heatmap(im, valfmt=format_str)

fig.tight_layout()
plt.show()

fig.savefig("totals.pdf", bbox_inches='tight')

In [ ]:
'''
Confusion Matrix
'''

rows = df_conf.index
cols =  df_conf.columns

metric = "Count"

format_str = "{x:}"

values = df_conf.values.astype(int)



fig, ax = plt.subplots()
ax.grid(False)

im, cbar = heatmap(values, rows, cols, ax=ax,
                   cmap="Blues", cbarlabel=metric)
texts = annotate_heatmap(im, valfmt=format_str)

fig.tight_layout()
plt.show()

fig.savefig("Conf_mat.pdf", bbox_inches='tight')